# 言語処理100本ノック
# 第5章: 係り受け解析

---

夏目漱石の小説『吾輩は猫である』の文章（`neko.txt`）を`CaboCha`を使って係り受け解析し，その結果を`neko.txt.cabocha`というファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

- `Cabocha` : [レファレンス](http://chasen.naist.jp/chaki/t/2005-08-29/doc/CaboCha%20Yet%20Another%20Japanese%20Dependency%20Structure%20Analyzer.htm)

>Support Vector Machines (SVMs) に基づく, 高性能な係り受け解析器  
>SVM の分類アルゴリズムの高速化手法である PKE (ACL 2003 にて発表)を適用.  
>決定的な解析アルゴリズム (Cascaded Chunking Model) を採用. 高効率な解析  
>係り関係そのものを素性として考慮する「動的素性」を採用, 精度向上に大きく貢献  
>文節の区切にも SVMs を採用 (実際には YamChaを使用)  
>単純な並列/同格構造解析が可能  
>IREX の定義による固有表現解析が可能  
>柔軟な入力形式. 生文はもちろん, 形態素解析済みデータ, 文節区切り済み データ, 部分的に係り関係が付与された>データからの解析が可能  
>係り受けの同定に使用する素性をユーザ側で再定義可能  
>データを用意すれば, ユーザ側で学習を行うことが可能  
>内部の辞書に, 高速な Trie 構造である Double-Array を採用  
>1文/秒程度の現実的な解析速度  
>C/C++/Perl/Ruby ライブラリの提供  

- [Cabocha Documentation](http://taku910.github.io/cabocha/)

In [1]:
from tqdm import tqdm_notebook as tqdm
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

- lattice（CABOCHA_FORMAT_LATTICE） 型で解析を行う

In [3]:
if os.path.exists("neko.txt.cabocha"):
    print("neko.txt.cabocha has already been exist")
else:
    os.system("cabocha -f1 neko.txt > neko.txt.cabocha")

neko.txt.cabocha has already been exist


- `Cabocha`のインストール (必要ならば)

In [4]:
# % brew install cabosha

## 40. 係り受け解析結果の読み込み（形態素）

---

形態素を表すクラス`Morph`を実装せよ．このクラスは表層形（`surface`），基本形（`base`），品詞（`pos`），品詞細分類1（`pos1`）をメンバ変数に持つこととする．さらに，**CaboCha**の解析結果（`neko.txt.cabocha`）を読み込み，**各文をMorphオブジェクトのリスト**として表現し，3文目の形態素列を表示せよ．

In [5]:
# 形態素を表すクラス`Morph`を実装せよ
class Morph:

    def __init__(self, surface : str, base : str,
                 pos : str, pos1 : str) -> None:

        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

        return None


    def get_list(self) -> list:
        return [self.surface, self.base , self.pos , self.pos1]

    def get_dict(self) -> dict:
        return {"surface":self.surface, "base":self.base ,
                "pos":self.pos , "pos1":self.pos1}


- `neko.txt.cabocha`の解析結果

In [6]:
%%bash
head -20 neko.txt.cabocha

EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
* 2 -1D 0/0 0.000000
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。


- 実装

In [7]:
# CaboChaの解析結果（neko.txt.cabocha）を読み込み
with open('neko.txt.cabocha', encoding='utf-8') as f:
    text_sentences = f.read().rstrip().split("EOS")
    text_sentences = [[morphs.split("\t") for morphs in sentence.split("\n")] for sentence in text_sentences]


# 各文をMorphオブジェクトのリストとして表現し
sentences = list()
for sentence in tqdm(text_sentences):
    sentence_morph = list()
    for line in sentence:
        if line[0] != "":
            if line[0][0] != ("*" or " " or "\u3000"):
                ## Line Example
                # [[生れ], [動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ]]
                surface = line[0]
                others = line[1].split(",")
                base = others[6]
                pos = others[0]
                pos1 = others[1]
                sentence_morph.append(Morph(surface, base, pos, pos1))

    if sentence_morph != []:
        sentences.append(sentence_morph)

In [8]:
def get_morph_list(sentences: list, idx: int) -> list:

    assert idx < len(sentences), "input index is larger than length of sentences"
    
    result = []
    for morph in sentences[idx - 1]:
        result.append(morph.get_list())

    return result


# 3文目の形態素列を表示せよ
get_morph_list(sentences, 3)

[['\u3000', '\u3000', '記号', '空白'],
 ['どこ', 'どこ', '名詞', '代名詞'],
 ['で', 'で', '助詞', '格助詞'],
 ['生れ', '生れる', '動詞', '自立'],
 ['た', 'た', '助動詞', '*'],
 ['か', 'か', '助詞', '副助詞／並立助詞／終助詞'],
 ['とんと', 'とんと', '副詞', '一般'],
 ['見当', '見当', '名詞', 'サ変接続'],
 ['が', 'が', '助詞', '格助詞'],
 ['つか', 'つく', '動詞', '自立'],
 ['ぬ', 'ぬ', '助動詞', '*'],
 ['。', '。', '記号', '句点']]

## 41. 係り受け解析結果の読み込み（文節・係り受け）

---

40に加えて，文節を表すクラス`Chunk`を実装せよ．このクラスは形態素（`Morph`オブジェクト）のリスト（`morphs`），係り先文節インデックス番号（`dst`），係り元文節インデックス番号のリスト（`srcs`）をメンバ変数に持つこととする．さらに，入力テキストの`CaboCha`の解析結果を読み込み，１文を`Chunk`オブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [10]:
"""
文節を表すクラスChunkを実装せよ．
このクラスは形態素（Morphオブジェクト）のリスト（morphs），
係り先文節インデックス番号（dst），
係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする
"""
class Chunk:
    
    def __init__(self, morphs : list, dst : int, srcs : list) -> None:

        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

        return None

    def get_list(self) -> list:
        return [self.morphs ,self.dst ,self.srcs]

    def get_dict(self) -> dict:
        return {"morphs" : self.morphs,
                "dst" : self.dst,
                "srcs" : self.srcs}


In [13]:
"""
入力テキストのCaboChaの解析結果を読み込み，
１文をChunkオブジェクトのリストとして表現し，
8文目の文節の文字列と係り先を表示せよ
"""

text_sentences

[[['']],
 [[''],
  ['* 0 2D 0/0 -0.764522'],
  ['\u3000', '記号,空白,*,*,*,*,\u3000,\u3000,\u3000'],
  ['* 1 2D 0/1 -0.764522'],
  ['吾輩', '名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ'],
  ['は', '助詞,係助詞,*,*,*,*,は,ハ,ワ'],
  ['* 2 -1D 0/2 0.000000'],
  ['猫', '名詞,一般,*,*,*,*,猫,ネコ,ネコ'],
  ['で', '助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ'],
  ['ある', '助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル'],
  ['。', '記号,句点,*,*,*,*,。,。,。'],
  ['']],
 [[''],
  ['* 0 2D 0/1 -1.911675'],
  ['名前', '名詞,一般,*,*,*,*,名前,ナマエ,ナマエ'],
  ['は', '助詞,係助詞,*,*,*,*,は,ハ,ワ'],
  ['* 1 2D 0/0 -1.911675'],
  ['まだ', '副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ'],
  ['* 2 -1D 0/0 0.000000'],
  ['無い', '形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ'],
  ['。', '記号,句点,*,*,*,*,。,。,。'],
  ['']],
 [[''], ['']],
 [[''],
  ['* 0 1D 1/2 1.058678'],
  ['\u3000', '記号,空白,*,*,*,*,\u3000,\u3000,\u3000'],
  ['どこ', '名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ'],
  ['で', '助詞,格助詞,一般,*,*,*,で,デ,デ'],
  ['* 1 4D 0/2 -1.453749'],
  ['生れ', '動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ'],
  ['た', '助動詞,*,*,*,特殊・タ,基本形,た,タ,タ'],
  ['か', '助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ'],
  ['

## 42. 係り元と係り先の文節の表示

---

係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．


In [11]:
who

Chunk	 Morph	 base	 f	 get_morph_list	 line	 os	 others	 pd	 
plt	 pos	 pos1	 sentence	 sentence_morph	 sentences	 surface	 sys	 text_sentences	 
tqdm	 


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

---

名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．


## 44. 係り受け木の可視化

---

与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．


## 45. 動詞の格パターンの抽出

---

今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```
始める  で
見る    は を
このプログラムの出力をファイルに保存し，
以下の事項をUNIXコマンドを用いて確認せよ．
```

コーパス中で頻出する述語と格パターンの組み合わせ
「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


## 46. 動詞の格フレーム情報の抽出

---

45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```
始める  で      ここで
見る    は を   吾輩は ものを
```

## 47. 機能動詞構文のマイニング

---

動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．

```
返事をする      と に は        及ばんさと 手紙に 主人は
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
```

コーパス中で頻出する述語（サ変接続名詞+を+動詞）
コーパス中で頻出する述語と助詞パターン


## 48. 名詞から根へのパスの抽出

---

文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

各文節は（表層形の）形態素列で表現する
パスの開始文節から終了文節に至るまで，各文節の表現を`"->"`で連結する
「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
```

## 49. 名詞間の係り受けパスの抽出

---

文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する


- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する


また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示


- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を"|"で連結して表示


例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y
```